In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader



In [2]:
pwd

'/content'

In [3]:
# loading data from google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
file = 'drive/My Drive/Data_precog/fmnist_small.csv'

In [5]:
import pandas as pd
import numpy as np
df = pd.read_csv(file)

In [6]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [7]:
number_of_training_examples = df.shape[0]
print(number_of_training_examples)

6000


In [8]:
number_of_features = df.shape[1]
print(number_of_features)

785


In [9]:
from sklearn.model_selection import train_test_split


# train , test split and x and y split
X = df.iloc[:,1:]
y = df.iloc[:,0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = X_train/ 255.0
X_test = X_test/ 255.0

In [10]:
# converting everything to tensors
X_train = torch.tensor(X_train.values , dtype = torch.float32)
X_test = torch.tensor(X_test.values, dtype = torch.float32)
y_train = torch.tensor(y_train.values , dtype = torch.long)
y_test = torch.tensor(y_test.values , dtype = torch.long)

In [11]:
# setting up the dataset from torch.utils.data
class CustomDataset(Dataset) :
  def __init__(self, features, labels) :
    self.features = features
    self.labels = labels

  def __len__(self) :
    return len(self.features)

  def __getitem__(self, idx) :
    return self.features[idx], self.labels[idx]


In [12]:
training_dataset = CustomDataset(X_train, y_train)
testing_dataset = CustomDataset(X_test, y_test)

In [13]:
# setting up the dataloaders
train_dataloader = DataLoader(training_dataset, batch_size = 64, shuffle = True)
test_dataloader = DataLoader(testing_dataset, batch_size = 64, shuffle = False) # no need to shuffle for test

In [14]:
# time to set up the model
class Model (nn.Module) :
  def __init__(self, num_of_features) : # it will be 784
    super().__init__()
    self.network = nn.Sequential(
        nn.Linear(num_of_features, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 10) ,
      #  nn.Softmax(dim = 1) # it is a multiclass classification problem : no need to explicitly mentioned
    )

  def forward (self, features) :
      return self.network(features)


In [15]:
X_train.shape[1]

784

In [16]:
!pip install torchinfo

model = Model(X_train.shape[1]) # object of model created
from torchinfo import summary

In [17]:
summary(model, input_size=(X_train.shape[0], X_train.shape[1]))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [4800, 10]                --
├─Sequential: 1-1                        [4800, 10]                --
│    └─Linear: 2-1                       [4800, 128]               100,480
│    └─ReLU: 2-2                         [4800, 128]               --
│    └─Linear: 2-3                       [4800, 64]                8,256
│    └─ReLU: 2-4                         [4800, 64]                --
│    └─Linear: 2-5                       [4800, 10]                650
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
Total mult-adds (M): 525.05
Input size (MB): 15.05
Forward/backward pass size (MB): 7.76
Params size (MB): 0.44
Estimated Total Size (MB): 23.25

In [18]:
learning_rate = 0.1
epochs = 100
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [ ]:
for epoch in range(0, epochs) :
  total_loss = 0
  for batch_features, batch_labels in train_dataloader :
    # forward pass
    y_pred = model(batch_features)

    # loss calculation
    loss = loss_fn(y_pred, batch_labels)
    # clearing gradients
    optimizer.zero_grad()

    # backpropagation
    loss.backward()

    # updating the parameters
    optimizer.step()
    total_loss += loss

  print(epoch, total_loss / len(train_dataloader))



In [21]:
# doing evaluation
model.eval() # important : to make sure not to do batch normalization , dropout

Model(
  (network): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [24]:
total = 0
correct = 0


with torch.no_grad() :
  for batch_features, batch_labels in test_dataloader :
    output = model(batch_features)
    _, predicted = torch.max(output , dim = 1)
    total = total + batch_labels.size(0)
    correct = correct + (predicted == batch_labels).sum().item()

print("Accuracy is ")
print(correct / total)

Accuracy is 
0.835
